# Read Result

In [ ]:
import pandas as pd 
# wannaone
input_data=pd.read_excel(r'predicted_101_twit.xlsx')

In [ ]:
channel = input_data['channel'].as_matrix()
## date = input_data['date'].as_matrix()
contents = input_data['contents'].as_matrix()
point = input_data['point'].as_matrix()
label = input_data['predict_label'].as_matrix()

In [ ]:
count_pos = len(input_data.loc[input_data['predict_label'] == 'POS'])
count_neu = len(input_data.loc[input_data['predict_label'] == 'NEU'])
count_neg = len(input_data.loc[input_data['predict_label'] == 'NEG'])
count_all = len(input_data)
counts = [count_pos,count_neu,count_neg]
print("긍정: {}".format(count_pos), "비중: {:.2f}".format(count_pos/count_all))
print("중립: {}".format(count_neu), "비중: {:.2f}".format(count_neu/count_all))
print("부정: {}".format(count_neg), "비중: {:.2f}".format(count_neg/count_all))

# Composing Data...

In [ ]:
import re
def ex_keyword(h_x,keyword):
    temp = h_x
    for i in range(len(h_x)):
        try:
            temp[i] = temp[i].replace(keyword,"")
        except:
            temp[i] = str(temp[i])
            pass
    return temp

sub_contents = ex_keyword(contents,'워너원')
sub_contents = ex_keyword(sub_contents,'WANNAONE')

In [ ]:
from konlpy.tag import Twitter
from konlpy.tag import Hannanum
han = Hannanum()
twit = Twitter()

def tokenizer (doc):
    tagged = ['/'.join(t) for t in han.pos(doc)]
    result_words = [word for word in tagged if word.split('/')[-1] == 'N']
    result_nouns = [word.split('/')[0] for word in result_words if len(word.split('/')[0]) > 1]
    step_2 = ' '.join(result_nouns)
    tagged = ['/'.join(t) for t in twit.pos(step_2)]
    result_words = [word for word in tagged if word.split('/')[-1] == 'Noun']
    result_nouns = [word.split('/')[0] for word in result_words if len(word.split('/')[0]) > 1]
    step_3 = ' '.join(result_nouns)
    return step_3

In [ ]:
# point: 50 ~ 100

level_a = []
for i in range(len(sub_contents)):
    if point[i] >= 50:
        level_a.append(tokenizer(sub_contents[i]))

# point: -50 ~ 50
level_b = []
for i in range(len(sub_contents)):
    if point[i] >= -50 and point[i] < 50 :
        level_b.append(tokenizer(sub_contents[i]))

# point: -100 ~ -50
level_c = []
for i in range(len(sub_contents)):
    if point[i] < -50:
        level_c.append(tokenizer(sub_contents[i]))
doc_a = ''.join(level_a)
doc_b = ''.join(level_b)
doc_c = ''.join(level_c)
doc_all = [doc_a,doc_b,doc_c]

In [ ]:
# freq_1: 첫번째
def freq_1(level_b):
    doc_b = ''.join(level_b)
    frequency = {}
    text_string = doc_b
    match_pattern = re.findall(r'\b[가-힣]{2,10}\b', text_string)
    for word in match_pattern:
        count = frequency.get(word,0)
        frequency[word] = count + 1
    frequency_list = frequency.keys()
    sorted_freq_b = sorted(frequency.items(), key=operator.itemgetter(1),reverse=True)
    subset_b = []
    for i in range(0,20):
        subset_b.append(sorted_freq_b[i][0])
    return subset_b

# freq: 두번째
def freq_2(level_b):
    doc_b = ''.join(level_b)
    frequency = {}
    text_string = doc_b
    match_pattern = re.findall(r'\b[가-힣]{2,10}\b', text_string)
    for word in match_pattern:
        count = frequency.get(word,0)
        frequency[word] = count + 1
    frequency_list = frequency.keys()
    sorted_freq_b = sorted(frequency.items(), key=operator.itemgetter(1),reverse=True)
    subset_b = []
    for i in range(1,6):
        subset_b.append(sorted_freq_b[i][0])
    return subset_b

In [ ]:
def main(level_b):
    step_1 = freq_1(level_b)
    x=0
    box_b = []
    for i in step_1:
        box_b.append([])
        for j in level_b:
            if (i in j) and (len(j)>len(i)):
                box_b[x].append(j)
        x +=1
    rebox_b = []
    for i in range(len(box_b)):
        rebox_b.append(freq(box_b[i]))
    result = pd.DataFrame(data=rebox_b,index=step_1)
    return result

### LEVEL_A

In [ ]:
import re
import string
frequency = {}
text_string = doc_a
match_pattern = re.findall(r'\b[가-힣]{2,10}\b', text_string)
for word in match_pattern:
    count = frequency.get(word,0)
    frequency[word] = count + 1
     
frequency_list = frequency.keys()

import operator 
sorted_freq_a = sorted(frequency.items(), key=operator.itemgetter(1),reverse=True) 

In [ ]:
sorted_freq_a

### LEVEL_B

In [ ]:
import re
import string
frequency = {}
text_string = doc_b
match_pattern = re.findall(r'\b[가-힣]{2,10}\b', text_string)
for word in match_pattern:
    count = frequency.get(word,0)
    frequency[word] = count + 1
     
frequency_list = frequency.keys()

import operator 
sorted_freq_b = sorted(frequency.items(), key=operator.itemgetter(1),reverse=True) 

### LEVEL_C

In [ ]:
import re
import string
frequency = {}
text_string = doc_c
match_pattern = re.findall(r'\b[가-힣]{2,10}\b', text_string)
for word in match_pattern:
    count = frequency.get(word,0)
    frequency[word] = count + 1
     
frequency_list = frequency.keys()

import operator 
sorted_freq_c = sorted(frequency.items(), key=operator.itemgetter(1),reverse=True) 

In [ ]:
main(level_a)

In [ ]:
main(level_b)

In [ ]:
main(level_c)